In [36]:
import pandas as pd
import networkx as nx
import ipytest
from pathlib import Path

from typing import Dict, Callable
ipytest.autoconfig()

In [12]:
DATA_DIR = Path("../../../CDS-LANG/network_data")
assert DATA_DIR.exists()

[PosixPath('../../../CDS-LANG/network_data/R3.csv'), PosixPath('../../../CDS-LANG/network_data/2H6.csv'), PosixPath('../../../CDS-LANG/network_data/1H6.csv'), PosixPath('../../../CDS-LANG/network_data/1H4.csv'), PosixPath('../../../CDS-LANG/network_data/H8.csv'), PosixPath('../../../CDS-LANG/network_data/3H6.csv'), PosixPath('../../../CDS-LANG/network_data/2H4.csv'), PosixPath('../../../CDS-LANG/network_data/H5.csv'), PosixPath('../../../CDS-LANG/network_data/R2.csv'), PosixPath('../../../CDS-LANG/network_data/KJ.csv')]


In [15]:
all_files = list(DATA_DIR.glob("*.csv"))
test_df = pd.read_csv(all_files[0], sep="\t")

In [17]:
G = nx.from_pandas_edgelist(test_df, source="Source", target="Target", edge_attr=["Weight"])

- Save a CSV which shows the following for every node:
   - name; degree; betweenness centrality; eigenvector_centrality

In [60]:
%%ipytest
import numpy as np

def test_calc_measures():
    graph = nx.watts_strogatz_graph(10, 3, 0.3)
    measure_dict = {"degree_centrality": nx.degree_centrality, "betweenness_centrality": nx.betweenness_centrality}
    measure_df = calc_measures(graph, measure_dict)
    assert len(measure_df.columns) == 3
    assert "name" in measure_df.columns
    assert measure_df["degree_centrality"].min() > 0

def calc_measures(graph: nx.Graph, measure_dict: Dict[str, Callable]) -> pd.DataFrame:
    col_dict = {name: [] for name in measure_dict}
    for func_name, func in measure_dict.items():
        col_dict[func_name] = func(G)
    return pd.DataFrame.from_records(col_dict).reset_index().rename(columns={"index": "name"})

.                                                                                            [100%]
1 passed in 0.17s


In [81]:
from functools import partial
measure_dict = {"degree_centrality": nx.degree_centrality, 
                "betweenness_centrality": partial(nx.betweenness_centrality, weight="Weigth"), 
                 "eigenvector_centrality": partial(nx.eigenvector_centrality, weight="Weigth"),
               }

measure_df = calc_measures(G, measure_dict)

In [64]:
measure_df

,name,betweenness_centrality,degree_centrality,eigenvector_centrality
0,Clarence,0.015410,0.076923,0.067212
1,Richard,0.529803,0.507692,0.425085
2,Brakenbury,0.039261,0.123077,0.133008
3,Hastings,0.094631,0.261538,0.298027
4,Anne,0.030206,0.107692,0.157726
...,...,...,...,...
61,GhostAnne,0.028045,0.046154,0.005253
62,GhostBuckingham,0.044311,0.030769,0.043157
63,1Lord,0.000000,0.030769,0.050515
64,2Lord,0.000000,0.030769,0.050515


In [54]:
pd.DataFrame.from_records(col_dict).reset_index().rename(columns={"index": "name"})

,name,betweenness_centrality,degree_centrality
0,Clarence,0.015410,0.076923
1,Richard,0.529803,0.507692
2,Brakenbury,0.039261,0.123077
3,Hastings,0.094631,0.261538
4,Anne,0.030206,0.107692
...,...,...,...
61,GhostAnne,0.028045,0.046154
62,GhostBuckingham,0.044311,0.030769
63,1Lord,0.000000,0.030769
64,2Lord,0.000000,0.030769


# Visualization

In [66]:
from pyvis.network import Network

In [85]:
Gx = G.copy()

In [86]:
for name, func in measure_dict.items():
    nx.set_node_attributes(Gx, func(Gx), name)

In [72]:
nx.set_node_attributes(Gx, nx.betweenness_centrality(Gx), "betweenness_centrality")

In [90]:
def network_to_df(graph: nx.Graph) -> pd.DataFrame:
    return pd.DataFrame.from_dict(dict(graph.nodes(data=True)), orient="index") \
             .drop(columns=["size"]) \
             .reset_index() \
             .rename(columns={"index": "name"})

In [93]:
netdf = network_to_df(Gx)

In [120]:
netdf["eigenvector_centrality"].min()

2.4484629103079295e-09

In [ ]:
testy = {'Clarence': 0.07692307692307693,
 'Richard': 0.5076923076923077,
 'Brakenbury': 0.12307692307692308,
 'Hastings': 0.26153846153846155,
 'Anne': 0.1076923076923077}

min

In [111]:
%%ipytest
def test_degree_size():
    testy = {'Clarence': 1,
 'Richard': 0.5076923076923077,
 'Brakenbury': 0.12307692307692308,
 'Hastings': 0.26153846153846155,
 'Anne': 0.1}
    scaled_degrees = degree_size(testy, min_size=10)
    assert min(scaled_degrees.values()) == 10
    

def degree_size(degree_dict, min_size=10) -> Dict[str, "int"]:
    min_degree = min(degree_dict.values())
    max_degree = max(degree_dict.values())
    scale_factor = (max_degree / min_degree) * min_size
    return {k: v * scale_factor for k, v in degree_dict.items()}

def set_size_degree(graph, min_size=10):
    scaled_degrees = degree_size(nx.get_node_attributes(Gx, "degree_centrality"), min_size=min_size)
    nx.set_node_attributes(graph, scaled_degrees, "size")

.                                                                                            [100%]
1 passed in 0.01s


In [112]:
set_size(Gx)

In [115]:
nt = Network()
nt.from_nx(Gx)
nt.show("nx.html")